## 목표: “GPT처럼 다음 단어를 예측하는 Mini Transformer 만들기”

아래 코드는 한 파일 안에서 완전히 실행 가능한
가장 단순한 “딥러닝 기반 Mini GPT 구조”입니다.
(Colab / Jupyter 기준 약 2~3분 실행)

### 🧩 1. 데이터: 간단한 문장 학습
### 💬 요약 설명

| 단계 | 내용 | GPT 학습과의 관계 |
| --- | --- | --- |
| **1️⃣ 문장 준비** | “I love machine learning” 등 텍스트 데이터 | 문맥(Context) 제공 |
| **2️⃣ Tokenizer 학습** | 단어를 정수 ID로 변환 | 단어 단위의 임베딩 입력 |
| **3️⃣ 시퀀스 변환** | 문장을 [1,2,3,4] 형태로 | 모델 입력으로 사용 |
| **4️⃣ 다음 단계** | 이 시퀀스로부터 “다음 단어” 예측 학습 | GPT의 기본 원리(언어모델링) |

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, MultiHeadAttention, LayerNormalization, Dropout
from tensorflow.keras import Sequential

# ------------------------------------------------------------
# 🧩 예시 데이터
# GPT(Transformer 기반 언어모델)는 "이전 단어들을 보고 다음 단어를 예측"하는 방식으로 학습합니다.
# 아래 4개의 짧은 문장을 예시로 사용합니다.
# ------------------------------------------------------------
texts = [
    "I love machine learning",
    "machine learning is fun",
    "deep learning is powerful",
    "I love deep learning"
]

# ------------------------------------------------------------
# 🧠 토크나이저 (Tokenizer)
# 텍스트 데이터를 숫자(ID)로 바꾸기 위한 도구입니다.
# 1) 전체 단어들을 사전(vocabulary)으로 정리
# 2) 각 단어를 고유한 정수로 매핑
# ------------------------------------------------------------
tokenizer = tf.keras.preprocessing.text.Tokenizer()

# 학습할 단어들을 토크나이저에 등록
tokenizer.fit_on_texts(texts)

# 전체 단어(고유 단어)의 개수 계산
# word_index는 {단어: 정수} 딕셔너리 형태
vocab_size = len(tokenizer.word_index) + 1   # +1은 padding(0)을 위해 추가
print("Vocab size:", vocab_size)
# 예시 출력: Vocab size: 9 (단어 8개 + 패딩 1개)

# ------------------------------------------------------------
# ✨ 문장을 정수 시퀀스로 변환
# 'I love machine learning' → [1, 2, 3, 4] 와 같이 숫자열로 바뀝니다.
# 각 숫자는 단어의 인덱스를 의미합니다.
# ------------------------------------------------------------
seqs = tokenizer.texts_to_sequences(texts)

print("샘플 시퀀스:", seqs)
# 예시 출력:
# [[1, 2, 3, 4],
#  [3, 4, 5, 6],
#  [7, 4, 8, 9],
#  [1, 2, 7, 4]]
#
# 이 상태에서 모델은 예를 들어
# [1, 2, 3] → 다음 단어 4("learning")를 맞히는 식으로 학습하게 됩니다.


Vocab size: 9
샘플 시퀀스: [[2, 3, 4, 1], [4, 1, 5, 7], [6, 1, 5, 8], [2, 3, 6, 1]]


In [2]:
import numpy as np

# ------------------------------------------------------------
# 🧩 입력(input)과 정답(target) 시퀀스를 만들기 위한 빈 리스트
# ------------------------------------------------------------
input_seqs, target_seqs = [], []

# seqs = [[1, 2, 3, 4], [3, 4, 5, 6], ...] 와 같은 숫자 시퀀스
for seq in seqs:
    # 각 문장을 부분 시퀀스로 나누어 (이전 단어 → 다음 단어) 관계를 학습
    # 예: [1, 2, 3, 4]
    # → (입력, 타겟)
    #   [1] → 2
    #   [1, 2] → 3
    #   [1, 2, 3] → 4
    for i in range(1, len(seq)):
        input_seqs.append(seq[:i])  # i번째 이전까지를 입력으로
        target_seqs.append(seq[i])  # i번째 단어를 정답(다음 단어)으로

# ------------------------------------------------------------
# 🧮 가장 긴 시퀀스 길이 계산
# 짧은 문장들은 pad_sequences를 이용해 길이를 맞춰줍니다.
# ------------------------------------------------------------
maxlen = max(len(s) for s in input_seqs)

# 입력 데이터 패딩 (짧은 문장은 앞쪽에 0으로 채움)
X = tf.keras.preprocessing.sequence.pad_sequences(input_seqs, maxlen=maxlen)

# 타깃 단어를 원-핫 인코딩(one-hot encoding)
# 예: 4 → [0,0,0,1,0,0,0,0,0] (vocab_size 차원)
y = tf.keras.utils.to_categorical(target_seqs, num_classes=vocab_size)

# ------------------------------------------------------------
# 🔍 결과 확인
# X: (샘플 수, 시퀀스 길이)
# y: (샘플 수, 단어 개수) — 다음 단어를 맞히는 분류 문제!
# ------------------------------------------------------------
print("입력 시퀀스:", X.shape)
print("타겟 시퀀스:", y.shape)

# 예시 출력
# 입력 시퀀스: (12, 3)   → 12개의 훈련 샘플, 길이 3
# 타겟 시퀀스: (12, 9)   → 각 샘플의 다음 단어를 9차원 원-핫 벡터로 표현

입력 시퀀스: (12, 3)
타겟 시퀀스: (12, 9)


### Mini Transformer 블록 정의

> GPT는 Decoder 블록을 여러 개 쌓지만, 우리는 1층만 사용합니다.
> 

### 🧠 전체 구조 요약

| 블록 | 역할 | GPT 구조에서의 대응 |
| --- | --- | --- |
| **Embedding** | 단어를 벡터로 표현 | Token Embedding |
| **MultiHeadAttention** | 문맥 이해 | Self-Attention Layer |
| **LayerNorm + Residual** | 학습 안정화 | Transformer 표준 구성 |
| **Feed Forward Network** | 비선형 변환 | FFN 블록 |
| **Dropout** | 과적합 방지 | Regularization |
| **Dense(vocab_size, softmax)** | 다음 단어 확률 예측 | Output layer |

In [3]:
class MiniTransformer(tf.keras.Model):
    def __init__(self, vocab_size, embed_dim, num_heads, ff_dim):
        super().__init__()

        # ------------------------------------------------------------
        # 1️⃣ Embedding Layer
        # 단어 ID → 임베딩 벡터로 변환 (각 단어를 고정 길이의 벡터로 표현)
        # 예: 단어 1 → [0.12, -0.07, 0.55, ...]
        # ------------------------------------------------------------
        self.embedding = Embedding(vocab_size, embed_dim)

        # ------------------------------------------------------------
        # 2️⃣ Multi-Head Self-Attention
        # 입력 문장 내 모든 단어가 서로를 "참조"할 수 있도록
        # 문맥(Context)을 학습합니다.
        # num_heads: 병렬로 주의를 계산할 헤드의 개수
        # key_dim: 각 헤드의 차원
        # ------------------------------------------------------------
        self.attention = MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embed_dim
        )

        # ------------------------------------------------------------
        # 3️⃣ Feed Forward Network (FFN)
        # Attention으로 얻은 정보에 비선형 변환을 추가
        # Dense → ReLU → Dense 구조
        # ------------------------------------------------------------
        self.ffn = Sequential([
            Dense(ff_dim, activation='relu'),
            Dense(embed_dim)
        ])

        # ------------------------------------------------------------
        # 4️⃣ Layer Normalization
        # 각 단계를 정규화하여 학습을 안정화시킴
        # ------------------------------------------------------------
        self.layernorm1 = LayerNormalization()
        self.layernorm2 = LayerNormalization()

        # ------------------------------------------------------------
        # 5️⃣ Dropout
        # 과적합(overfitting) 방지를 위한 정규화 기법
        # ------------------------------------------------------------
        self.dropout = Dropout(0.2)

        # ------------------------------------------------------------
        # 6️⃣ 출력층
        # 각 단어가 “다음 단어”일 확률을 예측하는 Softmax 출력
        # 출력 차원 = 단어 개수(vocab_size)
        # ------------------------------------------------------------
        self.out = Dense(vocab_size, activation='softmax')


    def call(self, inputs):
        # ------------------------------------------------------------
        # 순전파(forward pass) 정의
        # ------------------------------------------------------------

        # 1️⃣ 단어 ID → 임베딩 벡터
        x = self.embedding(inputs)
        # (배치크기, 문장길이, 임베딩차원)

        # 2️⃣ Self-Attention
        attn_output = self.attention(x, x)  # (Q, K, V 모두 동일: 자기 자신 참조)

        # 3️⃣ 잔차 연결(residual) + 정규화
        x = self.layernorm1(x + attn_output)

        # 4️⃣ Feed Forward Network
        ffn_output = self.ffn(x)

        # 5️⃣ 잔차 연결 + 정규화
        x = self.layernorm2(x + ffn_output)

        # 6️⃣ Dropout 적용
        x = self.dropout(x)

        # 7️⃣ 마지막 단어 위치의 출력만 사용 (GPT는 마지막 토큰 예측)
        # x[:, -1, :] → 문장의 마지막 토큰의 벡터만 가져옴
        # Dense(vocab_size, softmax)로 “다음 단어” 확률 분포 생성
        return self.out(x[:, -1, :])


In [4]:
model = MiniTransformer(vocab_size, embed_dim=32, num_heads=2, ff_dim=64)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(X, y, epochs=200, verbose=1)
print("✅ 학습 완료!")


Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 3.1147
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.1667 - loss: 2.4457
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.5000 - loss: 1.9429
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.5000 - loss: 1.5770
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5833 - loss: 1.3123
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.5833 - loss: 1.1130
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.7500 - loss: 0.9598
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.7500 - loss: 0.8387
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.7500 - loss: 0.7415
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.8333 - loss: 0.6585
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.8333 - loss: 0.5869
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.8333 - 

### 💬 문장 생성 (GPT처럼 다음 단어 생성)

### 작동 과정 정리

| 단계 | 설명 | 예시 |
| --- | --- | --- |
| 1️⃣ 입력 문장 받기 | “I love” | 시드 문장 |
| 2️⃣ 숫자로 변환 | [1, 2] | Tokenizer 이용 |
| 3️⃣ 모델 예측 | 다음 단어 확률분포 | 예: [0.1, 0.2, 0.6, 0.1] |
| 4️⃣ argmax로 단어 선택 | 가장 확률 높은 단어 | “machine” |
| 5️⃣ 문장에 추가 | “I love machine” | 반복 수행 |
| 6️⃣ 완성 | “I love machine learning fun powerful ...” | 생성 결과 |

In [6]:
# ------------------------------------------------------------
# 🧩 인덱스 → 단어 매핑
# Tokenizer는 {단어: 번호} 형태로 저장되어 있기 때문에,
# 예측 결과(숫자)를 다시 단어로 바꾸기 위해 역매핑을 만듭니다.
# ------------------------------------------------------------
index_to_word = {v: k for k, v in tokenizer.word_index.items()}

# ------------------------------------------------------------
# 🧠 텍스트 생성 함수
# seed_text: 시작 문장 (예: "I love")
# max_words: 생성할 단어 수 (기본값 5)
# ------------------------------------------------------------
def generate_text(seed_text, max_words=5):
    for _ in range(max_words):
        # 1️⃣ 입력 문장을 숫자 시퀀스로 변환
        seq = tokenizer.texts_to_sequences([seed_text])[0]

        # 2️⃣ 학습 시 사용한 maxlen(문장 최대 길이)에 맞게 패딩
        seq = tf.keras.preprocessing.sequence.pad_sequences([seq], maxlen=maxlen)

        # 3️⃣ 현재 문맥(seq)을 모델에 입력 → 다음 단어의 확률 예측
        pred = np.argmax(model.predict(seq, verbose=0))
        # np.argmax: 확률이 가장 높은 단어의 인덱스를 선택

        # 4️⃣ 인덱스를 실제 단어로 변환
        next_word = index_to_word.get(pred, '')

        # 5️⃣ 예측된 단어를 문장에 이어붙임
        seed_text += ' ' + next_word

    # 6️⃣ 모든 단어를 생성한 후 완성된 문장 반환
    return seed_text

# ------------------------------------------------------------
# 🔍 예시: 두 개의 시작 문장으로 새 문장 생성
# ------------------------------------------------------------
print("🧩 생성 문장 예시:")
print(generate_text("I love", max_words=4))
print(generate_text("machine learning", max_words=4))


🧩 생성 문장 예시:
I love machine learning is fun
machine learning is fun deep learning


> 모든 단어가 나오는것은 확률싸움인것  
> 할루시네이션 발생가능성  
> 올바른 체계를 구성해야함

## 📘 실습 정리표

| 단계 | 개념 | 실습 코드 |
| --- | --- | --- |
| 1 | 단어를 숫자로 변환 (Tokenization) | `Tokenizer()` |
| 2 | 다음 단어 예측을 위한 데이터셋 구성 | Sliding window |
| 3 | Self-Attention 계산 | `MultiHeadAttention()` |
| 4 | 문장 생성 | 반복적으로 next word 예측 |